## Grupo 2: Análisis de reservas y cancelaciones hoteleras: patrones y predicciones
🎯 Objetivo del proyecto
Analizar los datos de reservas de hoteles para comprender el perfil de los clientes, identificar patrones de comportamiento (estacionalidad, duración de la estancia, precios) y estudiar los factores que influyen en las cancelaciones, utilizando herramientas de análisis de datos en Python.

### 1. Introducción y descripción del dataset

- Breve explicación del contexto del análisis

- Descripción de las variables disponibles

- Objetivo del estudio

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('hotel_bookings.csv')

### 2. Perfil general de las reservas (Patri)

Objetivo: Entender cómo son las reservas y los clientes en general.

Tareas:

* Limpieza y descripción del dataset: tipos de datos, valores nulos, duplicados.
* Análisis de variables descriptivas:

  * *Tipo de hotel* (city vs. resort).
  * *Número de huéspedes* (adultos, niños, bebés).
  * *Países de origen*: top 10 países.
  * *Canales de reserva* (agencias online, offline, directas).
* Visualizaciones:

  * Gráfico de barras comparando city hotel vs resort hotel.
  * Gráfico circular de canales de reserva.
  * Mapa mundial con procedencia de clientes.

### Preguntas a responder:

* ¿Cuál es el perfil típico de cliente?
* ¿Qué países aportan más reservas?
* ¿Por qué canales se reserva más?

In [ ]:
print("--- Primeras 5 filas ---")
print(df.head())

print("\n--- Información general y tipos de datos ---")
df.info()

print("\n--- Resumen estadístico de las columnas numéricas ---")
print(df.describe())

print("\n--- Cuántos datos nulos hay ---")
print(df.isnull().sum())

duplicados = df.duplicated()
print("Número de filas duplicadas:", duplicados.sum())

--- Primeras 5 filas ---
          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   arrival_date_week_number  arrival_date_day_of_month  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  ...  deposit_type  \
0                        0                     0       2  ..

### 3. Patrones de comportamiento y cancelaciones (Rodri)

Objetivo: Analizar cómo varían las reservas en función del tiempo, los precios y la estancia.

Tareas:

* Estudiar variables clave:

  * *Mes y año de llegada* → estacionalidad.
  * *Duración de la estadía* (noches de semana y fin de semana).
  * *Precio promedio por noche (adr)* → variación según temporada.
  * *Cancelaciones*: proporción general y por segmento.
* Visualizaciones:

  * Línea temporal con reservas por mes.
  * Boxplot de precios según temporada alta/baja.
  * Gráfico de barras: cancelaciones por canal de reserva.

### Preguntas a responder:

* ¿En qué meses hay más reservas y cancelaciones?
* ¿El precio influye en la probabilidad de cancelación?
* ¿Los hoteles de ciudad o de resort tienen más cancelaciones?


### 4. Predicción y recomendaciones (Albert)

Objetivo: Explorar qué factores están más relacionados con las cancelaciones y proponer conclusiones prácticas.

Tareas:

* Crear variables comparativas:

  * Reservas canceladas vs no canceladas.
  * Relación entre lead_time (antelación de reserva) y cancelación.
  * Clientes repetidores vs nuevos.
* Visualizaciones:

  * Heatmap de correlaciones entre variables numéricas y cancelación.
  * Gráfico de barras de cancelación según país.
  * Boxplot de lead_time para canceladas vs no canceladas.
* (Opcional avanzado) Entrenar un modelo simple de clasificación para predecir cancelaciones (ej: logistic regression, random forest).

### Preguntas a responder:

* ¿Qué variables son más importantes para explicar una cancelación?
* ¿Se pueden detectar patrones que ayuden a los hoteles a reducir cancelaciones?
* ¿Qué recomendaciones se pueden dar a un hotel basadas en el análisis?